In [2]:
import os
from PIL import Image
import piexif

In [4]:
# input_dir = os.getcwd()  # текущая директория
input_dir = os.getcwd() + '\\photos_new'  # текущая директория (для новых фото)
output_dir = os.path.join(input_dir, 'compressed_photos')  # сюда будут сохранены сжатые фото
os.makedirs(output_dir, exist_ok=True)

# Допустимые форматы
extensions = ('.jpg', '.jpeg', '.png', '.heic', '.heif')

# Коэффициент сжатия (чем меньше — тем сильнее сжимает)
QUALITY = 15  # от 1 до 95, оптимально 60–80

# Проверка поддержки HEIC
try:
    import pillow_heif
    pillow_heif.register_heif_opener()
except ImportError:
    print("⚠️ Для поддержки HEIC установите библиотеку: pip install pillow-heif")

### Рекурсивный обход всех подпапок и сжатие изображений

In [1]:
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith(extensions):
            src_path = os.path.join(root, file)

            # формируем путь назначения (внутри compressed_photos, с сохранением структуры подпапок)
            rel_path = os.path.relpath(src_path, input_dir)
            dst_path = os.path.join(output_dir, rel_path)
            img_name = dst_path.split('\\')[-1]

            # создаём промежуточные папки, если их нет
            dst_dir = os.path.dirname(dst_path)
            os.makedirs(dst_dir, exist_ok=True)

            try:
                # открываем и конвертируем
                img = Image.open(src_path)
                
                # пробуем извлечь EXIF (если есть)
                exif = img.info.get('exif')
                
                img = img.convert('RGB')

                # сохраняем как JPEG
                dst_jpg = os.path.splitext(dst_path)[0] + '_comp_flt.jpg'
                if exif:
                    img.save(dst_jpg, 'JPEG', quality=QUALITY, optimize=True, exif=exif)
                else:
                    img.save(dst_jpg, 'JPEG', quality=QUALITY, optimize=True)
                    print('У фото ', img_name, ' не сохранены метаданные')

                print(f"✅ Сжато: {src_path} → {dst_jpg}")

            except Exception as e:
                print(f"❌ Ошибка при обработке {src_path}: {e}")


NameError: name 'os' is not defined

### Создание geojson с точками фото

In [3]:
import os
import json
from PIL import Image, ExifTags
from GPSPhoto import gpsphoto

# === Настройки ===
# os.chdir(r"D:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos")
os.chdir(r"D:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU")
input_dir = os.getcwd()  # Рабочая директория
print(input_dir)
output_geojson = os.path.join(input_dir, "photos.geojson")

extensions = ('.jpg')

def get_gps_coords(img_path):
    """Извлекает GPS координаты из EXIF в десятичном формате."""
    data = gpsphoto.getGPSData(img_path)
    lat = data['Latitude']
    lon = data['Longitude']
    if not lat:
        return None

    # def to_decimal(val):
    #     # Если уже float, просто возвращаем
    #     if isinstance(val[0], float):
    #         d, m, s = val
    #         return d + m / 60 + s / 3600
    #     # Иначе предположим, что кортеж ((num,den),...)
    #     d = val[0][0] / val[0][1]
    #     m = val[1][0] / val[1][1]
    #     s = val[2][0] / val[2][1]
    #     return d + m / 60 + s / 3600

    # try:
    #     lat = to_decimal(gps_info["GPSLatitude"])
    #     lon = to_decimal(gps_info["GPSLongitude"])

    #     if gps_info.get("GPSLatitudeRef") == "S":
    #         lat = -lat
    #     if gps_info.get("GPSLongitudeRef") == "W":
    #         lon = -lon

    return lat, lon
    # except KeyError:
        # return None


# --- Основная логика ---
features = []

for root, _, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith(extensions):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, input_dir).replace("\\", "/")

            try:
                coords = get_gps_coords(file_path)

                if coords:
                    lat, lon = coords
                    features.append({
                        "type": "Feature",
                        "geometry": {"type": "Point", "coordinates": [lon, lat]},
                        "properties": {"photo": rel_path}
                    })
                    print(f"✅ {rel_path} — {lat:.6f}, {lon:.6f}")
                else:
                    print(f"⚠️ Нет геотега: {rel_path}")

            except Exception as e:
                print(f"❌ Ошибка {file_path}: {e}")

# --- Сохранение результата ---
geojson = {"type": "FeatureCollection", "features": features}

with open(output_geojson, "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"\n📍 Найдено {len(features)} фото с геотегами")
print(f"💾 GeoJSON сохранён в {output_geojson}")

d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU
✅ compressed_photos/IMG_0408_comp_flt.jpg — 60.753833, 46.314736
✅ compressed_photos/IMG_0442_comp_flt.jpg — 60.749108, 46.320697
✅ compressed_photos/IMG_0446_comp_flt.jpg — 60.748317, 46.322522
✅ compressed_photos/IMG_3800_comp_flt.jpg — 60.768081, 46.290292
✅ compressed_photos/IMG_3931_comp_flt.jpg — 60.768964, 46.314092
❌ Ошибка d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\IMG_3969_comp_flt.jpg: 'Latitude'
✅ compressed_photos/IMG_6181_comp_flt.jpg — 60.749919, 46.319089
✅ compressed_photos/IMG_8271_comp_flt.jpg — 60.758008, 46.310203
✅ compressed_photos/IMG_8312_comp_flt.jpg — 60.759364, 46.307397
✅ compressed_photos/IMG_8434_comp_flt.jpg — 60.755786, 46.310347
✅ compressed_photos/IMG_8634_comp_flt.jpg — 60.764075, 46.297497
✅ compressed_photos/IMG_8799_comp_flt.jpg — 60.764181, 46.301658
✅ compressed_photos/IMG_9460_comp_flt.jpg — 60.761033, 46.299319

📍 Найдено 12 фото с геот